### **6.10 - How to share data between your tasks with XComs**

Hasta ahora en el curso, no hemos visto cómo podemos compartir datos entre tareas en Airflow.  De hecho, si te acuerdas de los videos anteriores, incluso dije que Airflow no debe ser utilizado como una solución de streaming como Spark o Storm que sigue siendo cierto. Sin embargo, en algunos casos es posible que necesite compartir datos entre sus tareas y ahí es donde intervienen los XCOMs.  Pero antes de seguir adelante, empecemos con un caso de uso.  Supongamos que queremos obtener un registro de una tabla almacenada en una base de datos, realizar algunas comprobaciones y, a continuación, utilizar este registro para decidir qué tarea ejecutar a continuación.  ¿Cómo podemos hacerlo?  Una forma es compartir este registro con la tarea que se encargará de elegir la siguiente tarea utilizando XCOMs.  ¿Qué son los XCOMS? Las siglas XCOM significan "cross-communication" (comunicación cruzada) y permiten que tus tareas intercambien datos y estados entre ellas.  Esto puede ser de gran ayuda ya que obtienes un mayor control sobre tus tareas, pero también puede ser peligroso ya que puedes caer en la tentación de tomar Airflow como solución de data streaming.  ¿Cómo funcionan?  

<center><img src="https://i.postimg.cc/ZKCwH7Wz/a1044.png"></center>

Los XComs se definen mediante una key (clave), un value (valor) y un timestamp (marca de tiempo).  La key se define como una simple cadena, mientras que el value puede ser cualquier objeto que se pueda recoger.  Depende de ti asegurarte de utilizar objetos de tamaño apropiado. Para responder a la pregunta, ¿tienen los XCOM un límite de tamaño?  La respuesta es no, pero debe mantenerlos muy ligeros o puede correr el riesgo de ralentizar su data pipeline e incluso hacer que su instancia Airflow sea inestable.  No piense en compartir un dataframe de millones de filas entre sus tareas. Como mejor práctica, mantenga el valor pequeño, de lo contrario utilice un sistema externo como una base de datos para compartir datos intermedios. Por ejemplo, eso es lo que harás para implementar el patrón de intercambio de comprobación por etapas (stage-check-exchange pattern), en el que los datos se almacenan en una tabla temporal, tras lo cual se realizan comprobaciones de calidad de los datos en esa tabla para finalmente trasladar la partición a la tabla de producción.  Además de la key, value, timestamp. Se rellena el task id de la tarea que ha enviado el XCOM, así como el DAG id del que procede.  Esto permite recuperar un XCOM utilizando el task id o limitando la búsqueda de un XCOM a un DAG id específico.  Supongamos que queremos compartir datos entre dos operadores, ¿cómo podemos hacerlo?  Bueno, para empujar (push) un XCOM en la base de metadatos, hay dos maneras.  

<center><img src="https://i.postimg.cc/GmDqw4PM/a1045.png"></center>

Primero, desde el primer operador puedes llamar al método xcom_push. Este método espera una key y un valor como se muestra aquí.  Otra forma de empujar un XCom es devolviendo un valor desde el método execute de un Operator o desde la función python callable del PythonOperator.  En ese caso, el XCOM sólo devolverá el valor y la key se asignará automáticamente a la cadena "return_value". 

<center><img src="https://i.postimg.cc/pdtk2PqP/a1046.png"></center>

Una vez almacenado el XCOM en la base de metadatos, el segundo operador puede extraer este valor llamando al método “xcom_pull”.  Hay diferentes formas de extraer XComs. En primer lugar, puede especificar la key del XCOM empujado (pushed) que desee como se muestra en este ejemplo y estará disponible en su operador.  

<center><img src="https://i.postimg.cc/ZqFHnXMp/a1047.png"></center>

Otra forma es especificar sólo la task id de la tarea que ha empujado el XCOM que desea. 

<center><img src="https://i.postimg.cc/V6fRC4xT/a1048.png"></center>

Por ejemplo, digamos que la tarea 1 ha empujado un XCOM, entonces en la tarea 2 puede llamar al método xcom_pull con la task id “task 1” y obtendrá el XCOM más reciente empujado por la tarea 1. Por eso existe un timestamp junto con los XCOM. Para ordenarlos del más reciente al más antiguo. **`Tenga en cuenta que puede especificar varios task ids para obtener varios XCOM a la vez`**. En lugar de obtener un valor, obtendrá una tupla que contendrá los valores de sus XCOM en el mismo orden definido en el task ids parameter. Por último, si especifica varios task ids, no podrá establecer keys. Airflow intentará tirar (pull) de las tareas donde el XCOM ha sido empujado devolviendo el valor y no estableciendo un par clave-valor con el método xcom_push. Bien, sin más espera pasemos a la práctica.  